<a href="https://colab.research.google.com/github/TAYAyuki/adversarial_example/blob/master/Adversarial_examples0620.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd #データ解析

In [0]:
import keras
import tensorflow as tf

In [18]:
df=pd.read_csv("./mnist_test.csv")
labels=df["7"].values #正解ラベル -> labels
print("labels.shape : "+str(labels.shape)) # 1*99 の行列
one_hot_labels=tf.keras.utils.to_categorical(labels) #正解ラベル(0~9)をone_hot_vectorにする
print("one_hot_vector : "+str(one_hot_labels.shape)) #99*10の行列
df.drop("7", axis=1, inplace=True)  #正解ラベル削除
pixels=df.values #正解ラベル以外  -> pixels
pixels=pixels/255 #0 - 1の値に正規化
print("pixels_before reshape : "+str(pixels.shape))
pixels=pixels.reshape(len(df),28,28,1) #１つのpixelが28*28 白黒だから1 (カラーだったら3)
print("pixels.shape : "+ str(pixels.shape))

labels.shape : (9999,)
one_hot_vector : (9999, 10)
pixels_before reshape : (9999, 784)
pixels.shape : (9999, 28, 28, 1)


In [0]:
class Adversarial_Training(keras.Model):
    _at_loss = None
    # set up
    def setup_at_loss(self, loss_func=keras.losses.sparse_categorical_crossentropy, eps=0.25/255.0, alpha=1.0):
        self._loss_func = loss_func
        self._alpha = alpha
        self._at_loss = self.at_loss(eps)
        return self
      
    # VAT loss
    def at_loss(self, eps):
      
        # original loss
        loss_orig = self._loss_func(self.input[0], self.inputs[1])
        
        # gradients
        map_grads = keras.backend.stop_gradient(keras.backend.gradients(loss_orig, self.inputs[0]))
        list_grads = list(map_grads)
        grads = list_grads[0]
        
        # perterbed samples
        new_inputs = self.inputs[0] - eps * keras.backend.sign(grads)
        print("new_inputs", new_inputs)
        
        # estimation for the perturbated samples
        outputs_perturb = self.call([new_inputs, self.input[1]])
        print("outputs_perturb", outputs_perturb)
        
        # computing losses
        loss = self._loss_func(self.inputs[1], outputs_perturb)
        return loss

In [0]:
input_cnn=keras.layers.Input(shape=(28, 28, 1))
model_cnn=keras.layers.Conv2D(20, (3, 3), padding="valid", activation=tf.nn.relu,)(input_cnn)
model_cnn=keras.layers.MaxPooling2D(pool_size=(3,3))(model_cnn)
model_cnn=keras.layers.Conv2D(50, (3, 3), padding="valid", activation=tf.nn.relu,)(model_cnn)
model_cnn=keras.layers.MaxPooling2D(pool_size=(3,3))(model_cnn)
model_cnn=keras.layers.BatchNormalization()(model_cnn)
model_cnn=keras.layers.Flatten()(model_cnn)
model_cnn=keras.layers.Dense(225, activation=tf.nn.relu)(model_cnn)
model_cnn=keras.layers.Dense(10, activation="softmax")(model_cnn)


In [23]:
input_cnn_a=keras.layers.Input(shape=(10,))
model = Adversarial_Training([input_cnn, input_cnn_a], model_cnn).setup_at_loss(loss_func=keras.losses.categorical_crossentropy, eps=0.25, alpha=0.5)

new_inputs Tensor("sub_14:0", shape=(?, 28, 28, 1), dtype=float32)
outputs_perturb Tensor("dense_8_1/Softmax:0", shape=(?, 10), dtype=float32)


In [10]:
model.compile(optimizer=tf.train.AdamOptimizer(), loss="categorical_crossentropy", metrics=["accuracy"])

W0620 02:49:01.369701 139953292158848 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 26, 26, 20)        200       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 20)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 6, 50)          9050      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 2, 2, 50)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 2, 2, 50)          200       
_________________________________________________________________
flatten_1 (Flatten)          (None, 200)               0         
__________

In [12]:
target_number = 6
target_labels = np.zeros(one_hot_labels.shape)
target_labels[:, target_number] = 1
target_labels.shape


(9999, 10)

In [14]:
def input_derivative(model, x, goal):
   """ Calculate derivatives wrt the intputs """
   y_true = keras.layers.Input(shape=(10,))
   ce = keras.losses.categorical_crossentropy(y_true, model.output)
   grad_ce = tf.keras.backend.gradients(ce, model.inputs)
   # Create a function to be able to run this computation graph
   func = keras.backend.function(model.inputs + [y_true], grad_ce)
   output = func([x, goal])
   return output

def adversarial(model, n, steps, eta):
   """
   model: MNIST model
   n:     integer. goal label (just an int, the function transfroms it into a one-hot vector)
   steps: integer. number of steps for gradient descent
   eata:  integer. step size for gradient descent
   """
   # Set the goal output
   goal = tf.keras.utils.to_categorical(n, 10)
   # Create a random image to initialize gradient descent with
   x = np.random.normal(.5, .3, (1,784))
   # Gradient descent on the input
   for i in range(steps):
     # Calculate the derivative
     d = input_derivative(model, x, goal)
     # The Gradient update on x
     x -= eta*d[0]
     print("step:{} done...".format(i+1))
   return x

def generate(n):
   """
   n: integer
   goal label ( not hot vector)
   """
   a = adversarial(model, n, 100, 1)
   x = np.round(model.predict(a), 2)

   plt.imshow(a.reshape(28,28), cmap='gray')


def sneaky_adversarial(model, n, x_target, steps, eta, lam=0.5):
   """
   model:    model object
   n:        integer
        our toal label
   x_target: numpy vector
        our goal image for the adversarial example
   steps:    integer
        number of steps for gradient descent
   lam:      float
        lambda, our regularization parameter. Default is 0.5
   """

   # Set the goal output
   goal = tf.keras.utils.to_categorical(n, 10)
   
   # Create a random noise
   x = np.random.normal(0.5, 0.3, (1,784))

   
   # Gradient descent on the input
   for i in range(steps):
     # Calculate the derivative
     d = input_derivative(model, x, goal)
     # The grads update on x, with an added penalty to the cost function
     x -= eta * (d[0] + lam * (x - x_target))

   return x


def sneaky_generate(n, m):
   """
   n:    int 0-9, the target number to match
   m:    index of example image to use (from the test set)
   """
   # Find random instance of m in test set
   idx = np.random.randint(0,10000)
   while np.where(y_test[idx] == 1)[0][0] != m:
     idx += 1
   # Hardcode the parameters for the wrapper function
   a = sneaky_adversarial(model, n, x_test[idx], 10, 1)
   x = np.round(model.predict(a), 2)
   print('\nWhat we want our adversarial example to look like: ')
   plt.imshow(x_test[idx].reshape((28,28)), cmap='gray')
   plt.show()
   print('\n')
   print('Adversarial Example: ')
   plt.imshow(a.reshape(28,28), cmap='gray')
   plt.show()
   print('Network Prediction: ' + str(np.argmax(x)) + '\n')
   print('Network Output: \n' + str(x) + '\n')
   return a

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)
print(x_train.shape)
x_train = x_train.reshape(60000,28*28)
x_test = x_test.reshape(10000,28*28)
x_train = x_train/255.
x_test = x_test/255.
print(x_train.shape)


a = keras.layers.Input(shape=(28*28,))
x = keras.layers.Dense(units=512, activation='relu', input_shape=(28*28,))(a)
x = keras.layers.Dense(units=128, activation='relu')(x)
x = keras.layers.Dense(units=64, activation='relu')(x)
x = keras.layers.Dense(units=10, activation='softmax')(x)

model = keras.models.Model(inputs=a, outputs=x)
model.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['accuracy'])
#model.fit(x_train, y_train, batch_size=64, epochs=5, validation_data=(x_test, y_test))

"""
a = keras.layers.Input(shape=(28*28,))
x = keras.layers.Conv2D(20, (3, 3), padding="valid", activation=tf.nn.relu,)(a)
x = keras.layers.MaxPooling2D(pool_size=(3,3))(x)
x = keras.layers.Conv2D(50, (3, 3), padding="valid", activation=tf.nn.relu,)(x)
x = keras.layers.MaxPooling2D(pool_size=(3,3))(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(225, activation=tf.nn.relu)(x)
x = keras.layers.Dense(10, activation="softmax")(x)
"""

# sneaky_generate(target label, target digit)
adv_ex = sneaky_generate(8, 2)

11493376/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)
(60000, 784)

What we want our adversarial example to look like: 


NameError: ignored

In [13]:
fit=model.fit([pixels,target_labels],one_hot_labels,epochs=50, batch_size=4096,validation_split=0.2)

W0620 02:49:08.767678 139953292158848 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 7999 samples, validate on 2000 samples
Epoch 1/50


KeyboardInterrupt: ignored